# Utils

In [1]:
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from feature_selection_functions import amostragem, remove_highly_correlated_features, generate_metadata, variancia, vars_selection
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("FeatureEng") \
    .config("spark.executor.memory", "14g") \
    .config("spark.driver.memory", "14g") \
    .getOrCreate()
from warnings import filterwarnings
filterwarnings('ignore')
from random import seed
seed(1)

In [2]:
# Função para criar lotes de colunas
def criar_lotes(lista, tamanho):
    
    qtd = len(lista) // tamanho
    residuo = len(lista) % tamanho
    lotes = []
    slice_start = 0
    slice_end = tamanho

    for _ in range(1,qtd+2):
        if _ == (qtd+2):
            lotes.append(lista[len(lista) - residuo:len(lista)-1])
            break

        lotes.append(lista[slice_start:slice_end])
        slice_start+= tamanho
        slice_end+= tamanho
    
    return lotes

# Loading

* Carregando os Dados

In [3]:
# Tabela com os dados do Bureau, após feature engineering
bureau = spark.read.parquet('./BASES_TREINO_TESTE/BUREAU_FEAT_ENG')

# Tabela com os dados de Previous Application (POSCASH Balance, Instalments Payments e Credit Card Balance)
# Após feat. eng.
prev_app = spark.read.parquet('./BASES_TREINO_TESTE/PREV_APP_AGG_FEAT_ENG')

# Base de Treino (Application Train)
app_train = spark.read.csv('./DATASETS/application_train.csv', header= True, inferSchema= True)

* Fazendo o Join

In [4]:
app_train_final = app_train.join(other= prev_app, on= "SK_ID_CURR", how= 'left').join(other= bureau, on= "SK_ID_CURR", how= 'left')
app_train_final.count(), len(app_train_final.columns)

(215257, 10964)

* Liberando Memória

In [5]:
from gc import collect
del(bureau, prev_app, app_train)
collect()

21

# Feature Selection

* Aqui irei realizar a seleção de variáveis em lotes, em vista do tamanho da tabela final e das limitações computacionais.

* Salvando o nome das colunas da tabela final

In [6]:
# Pegando todas as colunas da tabela final
app_train_final_cols = app_train_final.columns

app_train_final_cols.remove('SK_ID_CURR')
app_train_final_cols.remove('TARGET')

# Verificando o tamanho final
len(app_train_final_cols)

10962

* Pegando um recorte da tabela final

In [22]:
lotes = criar_lotes(app_train_final_cols, 500)

variaveis_selecionadas = []

for lote in tqdm(lotes):
    lote.append('SK_ID_CURR')
    lote.append('TARGET')

    temp01 = app_train_final.select(*lote).toPandas()

    selecao = vars_selection(temp01, percentual_preenchimento= 80, threshold= 0.5, tamanho_amostragem= 90000)

    if not selecao.empty:
        for variavel in selecao['Variável']:
            variaveis_selecionadas.append(variavel)
    else:
        pass
    print("Vars Selecionadas:", len(variaveis_selecionadas))

  0%|          | 0/22 [00:00<?, ?it/s]

  5%|▍         | 1/22 [07:14<2:32:04, 434.49s/it]

Vars Selecionadas: 93


  9%|▉         | 2/22 [07:38<1:04:23, 193.16s/it]

Vars Selecionadas: 95


...

Vars Selecionadas: 203


 95%|█████████▌| 21/22 [15:20<00:25, 25.91s/it]

Vars Selecionadas: 208


100%|██████████| 22/22 [16:13<00:00, 44.23s/it]

Vars Selecionadas: 218


* Aqui conseguimos passar de 10900 variáveis para 218!

In [25]:
# Adicionando novamente o ID e o Target
variaveis_selecionadas.append('TARGET')
variaveis_selecionadas.append('SK_ID_CURR')

In [30]:
app_train_final_selected = app_train_final.select(*variaveis_selecionadas)

In [31]:
# Salvando a Application Train Final com as Vars. Selecionadas
app_train_final_selected = app_train_final_selected.repartition(1)
app_train_final_selected.write.parquet('./BASES_TREINO_TESTE/APPLICATION_TRAIN_FINAL')

In [33]:
# Vendo o tamanho da Tabela
app_train_final_selected.count(), len(app_train_final_selected.columns)

(215257, 220)

In [36]:
with open('./ARTEFATOS/vars_selecionadas_apptrain.pickle', 'wb') as arquivo:
    pickle.dump(variaveis_selecionadas, arquivo)

* Fazendo o mesmo para a Application Test

In [2]:
# Tabela com os dados do Bureau, após feature engineering
bureau = spark.read.parquet('./BASES_TREINO_TESTE/BUREAU_FEAT_ENG')

# Tabela com os dados de Previous Application (POSCASH Balance, Instalments Payments e Credit Card Balance)
# Após feat. eng.
prev_app = spark.read.parquet('./BASES_TREINO_TESTE/PREV_APP_AGG_FEAT_ENG')

# Base de Teste (Application Train)
app_test = spark.read.csv('./DATASETS/application_test.csv', header= True, inferSchema= True)

In [16]:
with open('./ARTEFATOS/vars_selecionadas_apptrain.pickle', 'rb') as arquivo:
    variaveis_selecionadas = pickle.load(arquivo)

In [5]:
app_test_final = app_test.join(other= prev_app, on= "SK_ID_CURR", how= 'left').join(other= bureau, on= "SK_ID_CURR", how= 'left')
app_test_final.count(), len(app_test_final.columns)

(92254, 10963)

* Filtrando apenas as variáveis selecionadas

In [17]:
variaveis_selecionadas.remove('TARGET')

app_test_final_selected = app_test_final.select(*variaveis_selecionadas)

app_test_final_selected = app_test_final_selected.repartition(1)

app_test_final_selected.write.parquet('./BASES_TREINO_TESTE/APPLICATION_TEST_FINAL')